# Separation

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial05-setpartition.ipynb)


In [ ]:
# If using this in Google Colab, we need to install the required packages
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !pip install eqc-models==0.12.0 # The version may change, check the latest on PyPI

In [1]:
import numpy as np
from eqc_models.combinatorics import SetPartitionModel
from eqc_models.solvers import Dirac3IntegerCloudSolver

In [2]:
S = [{"A", "B"}, {"A", "C"}, {"B"}, {"C"}]
weights = [4, 4, 1, 3]

In [3]:
model = SetPartitionModel(S, weights)
model.penalty_multiplier = 10

In [4]:
solver = Dirac3IntegerCloudSolver()

In [5]:
response = solver.solve(model, relaxation_schedule=1, num_samples=5)

2025-03-14 23:07:36 - Dirac allocation balance = 0 s (unmetered)
2025-03-14 23:07:36 - Job submitted: job_id='67d50b1700e804f113aefcae'
2025-03-14 23:07:36 - QUEUED
2025-03-14 23:07:38 - RUNNING
2025-03-14 23:07:44 - COMPLETED
2025-03-14 23:07:46 - Dirac allocation balance = 0 s (unmetered)


In [6]:
solution = response["results"]["solutions"][0]
U = set()

for i in range(len(solution)):
    if solution[i] == 1:
        U = U.union(S[i])
solution, U

([0, 1, 1, 0], {'A', 'B', 'C'})